In [11]:
import pandas as pd
import numpy as np
import pybaseball as pyb
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt, atan2, degrees, pi
from scipy.stats import percentileofscore

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
data_2023 = pyb.statcast(start_dt='2023-03-30', end_dt='2023-10-01')

This is a large query, it may take a moment to complete


/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|█████████████████████████████████████████| 186/186 [00:38<00:00,  4.82it/s]


In [3]:
fastballs = data_2023[data_2023['pitch_type'].isin(['FF'])]

In [4]:
fastballs.dropna(subset=['vy0'], inplace=True)

/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [5]:
def calculate_vaa(row):
    yf = 17/12  # Home plate distance in feet, converted to inches
    ay = row['ay']  # Acceleration in y-dimension for the current pitch
    vy0 = row['vy0']  # Velocity in y-dimension at y=50 feet for the current pitch
    vz0 = row['vz0']  # Velocity in z-dimension at y=50 feet for the current pitch
    az = row['az']  # Acceleration in z-dimension for the current pitch

    vy_f = -sqrt(vy0**2 - (2 * ay * (50 - yf)))
    t = (vy_f - vy0) / ay
    vz_f = vz0 + (az * t)
    vaa_rad = atan2(vz_f, vy_f)
    vaa_deg = (180+degrees(vaa_rad))*-1

    return vaa_deg
fastballs['VAA'] = fastballs.apply(calculate_vaa, axis=1)

/var/folders/bf/7kjr62fs6wq3x9hlt0xsr0w40000gn/T/ipykernel_33245/716655449.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastballs['VAA'] = fastballs.apply(calculate_vaa, axis=1)


In [6]:
filtered_ff = fastballs.loc[:, ['release_pos_z', 'player_name', 'pfx_z', 'release_spin_rate','plate_x','spin_axis','VAA','release_extension','zone','plate_z','release_speed','release_pos_x','release_pos_y','vx0','vy0','vz0']]

In [7]:
filtered_ff['VRA'] = np.arctan(filtered_ff['vz0'] / np.sqrt(filtered_ff['vx0']**2 + filtered_ff['vy0']**2))
filtered_ff['HRA'] = np.arctan(filtered_ff['vx0'] / filtered_ff['vy0'])
filtered_ff['VRA_degrees'] = np.degrees(filtered_ff['VRA'])
filtered_ff['HRA_degrees'] = (np.degrees(filtered_ff['HRA']))*-1

In [8]:

pitch_count = filtered_ff['player_name'].value_counts()

filtered_ff_filtered = filtered_ff[filtered_ff['player_name'].isin(pitch_count.index[pitch_count >= 500])]


In [9]:
import pandas as pd
import numpy as np


pitcher_std = filtered_ff_filtered.groupby('player_name')[['VRA_degrees', 'HRA_degrees','release_pos_z','release_pos_x']].std()


pitcher_std['tightness_metric'] = pitcher_std.mean(axis=1)

ranked_pitchers = pitcher_std.sort_values('tightness_metric')

In [10]:
ranked_pitchers.head()

,VRA_degrees,HRA_degrees,release_pos_z,release_pos_x,tightness_metric
player_name,,,,,
"Sewald, Paul",0.615667,0.817073,0.090110,0.083528,0.401595
"Kirby, George",0.713257,0.691793,0.103670,0.137947,0.411667
"Ober, Bailey",0.774295,0.630723,0.100591,0.142407,0.412004
"Ryan, Joe",0.749711,0.675591,0.153881,0.109306,0.422122
"Bradford, Cody",0.810903,0.649964,0.095183,0.142762,0.424703


In [14]:
X = filtered_ff[['HRA_degrees', 'VRA_degrees','release_pos_x','release_pos_z']]
y = filtered_ff[['plate_x', 'plate_z']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 0.06128000089206968
R-squared: 0.8982042650735885


/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.2

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rf_regressor = RandomForestRegressor()

rf_regressor.fit(X_train, y_train)

feature_importances = rf_regressor.feature_importances_

y_pred = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print("Mean Absolute Error:", mae)

for i, importance in enumerate(feature_importances):
    print(f"Feature {i+1}: {importance}")


Mean Absolute Error: 0.18427725070300816
Feature 1: 0.27071017380266765
Feature 2: 0.4187379131370986
Feature 3: 0.17089519243517567
Feature 4: 0.13965672062505824


/Users/michaelrosen/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
